# Default value study

Goal: calculate minima and store these per feature

- a1 (jet_pt) - no default value necessary, all jets have some pt
- a2 (jet_eta) - no default value necessary, all jets have some eta
- a3 (flavor) - no default value necessary, all jets have some flavor
- a4 (high_level_track) - 8 variables
    - defaults previously:
    ```c++
      track2d0sig = -inf;
      track2z0sig = -inf;
      track3d0sig = -inf;
      track3z0sig = -inf;
      tracksOverIpThreshold = 0;
      jetProb = -1;
      jetWidthEta = -inf;
      jetWidthPhi = -inf;
      ```
    - defaults for plotting by me in case of nan, -inf, +inf:
        - -999
- a5 (high_level_vertex) - 6 variables
    - defaults previously:
    ```c++
      Lsig = has_vx ? sum_sig / sqrt(sum_inverr) : -1;
      NVertex = has_vx ? sum_vertices               : -1;
      NTracks = has_vx ? sum_tracks                 : -1;
      DrJet = has_vx ? sum_dr_tracks / sum_tracks : inf;
      Mass = has_vx ? sum_mass                   : -1;
      EnergyFraction = has_vx ? sum_efrac : -inf;
    ```
    - defaults for plotting by me in case of nan, -inf, +inf:
        - -999
- a6 (low_level_track_vertex) - 924 variables
    - defaults previously (related to track-vertex):
        - -1
        - NaN
    - defaults for plotting by me in case of nan, -inf, +inf:
        - -1000
        
Definitions and source for these statements:  
http://mlphysics.ics.uci.edu/data/hb_jet_flavor_2016/readme.txt  
https://github.com/dguest/delphes-rave/wiki/Output-Format  
https://github.com/dguest/delphes-rave/blob/rave/classes/flavortag

In [1]:
import time

import awkward1 as ak
import numpy as np

import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import mplhep as hep

plt.style.use([hep.style.ROOT, hep.style.fira, hep.style.firamath])

In [3]:
user = 'um106329'
akArrays_path = '/hpcwork/' + user + '/jet_flavor_MLPhysics/dataset/akArrays/'
np_arrays_path = '/hpcwork/' + user + '/jet_flavor_MLPhysics/dataset/npArrays/'

In [109]:
defaults_path = '/hpcwork/' + user + '/jet_flavor_MLPhysics/dataset/defaults/'

In [4]:
splits = []
total = 11491971
for k in range(0,total,50000):
    splits.append(k)
splits.append(total)
len(splits)

231

In [5]:
array = ak.concatenate([ak.from_parquet(akArrays_path+'split_{0}_{1}.parquet'.format(splits[i],splits[i+1]-1)) for i in range(len(splits)-1)])

In [6]:
a1 = array.a1

In [7]:
a2 = array.a2

In [8]:
a3 = array.a3

In [9]:
a4 = array.a4

In [10]:
a5 = array.a5

In [11]:
import gc
del array
gc.collect()

256

## Investigate high-level variables

Some basics, should be more automatic later.

In [12]:
a1

<Array [47.9, 35, 26.6, ... 43.4, 25.9, 37.3] type='11491971 * ?float64'>

In [13]:
ak.min(a1)

22.9141

In [14]:
ak.max(a1)

884.806

In [15]:
np.unique(ak.to_numpy(a1))

masked_array(data=[ 22.9141,  22.9154,  22.9157, ..., 700.747 , 819.099 ,
                   884.806 ],
             mask=False,
       fill_value=1e+20)

In [16]:
# https://stackoverflow.com/questions/55193696/how-do-i-find-the-values-in-my-numpy-array-that-are-nan-infinity-too-large-for-d
np.where(~np.isfinite(ak.to_numpy(a1)), -1000, ak.to_numpy(a1))

array([47.8712, 34.9703, 26.5706, ..., 43.4182, 25.8566, 37.2836])

Import some helpful defintions related to indices and names of variables.

In [29]:
# https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import sys
sys.path.append("../helpers/")
from variables import *

In [32]:
# test if import worked
track_vertex_index_to_name(33)

'track_0_Z0'

In [81]:
n_high_level

17

In [193]:
def investigate_default(array, index, is_high_level, print_outputs=True):
    
    print(full_index_to_name(index))
    print()
    if is_high_level:
        original_min, original_max = ak.min(array), ak.max(array)
    else:
        original_min, original_max = np.min(array), np.max(array)
    if print_outputs:
        print('Min:\t{}\t\tMax:\t{}'.format(original_min, original_max))
    
    if is_high_level:
        replace_infs_nan = np.where(~np.isfinite(ak.to_numpy(array)), -999, ak.to_numpy(array))
    else:
        replace_infs_nan = np.where(~np.isfinite(array), -999, array)
    
    min_including_infs_nan = np.min(replace_infs_nan)
    if print_outputs:
        print('Min after replacing infs/nan: ', min_including_infs_nan)
    
    
    array_without_infs_nan = replace_infs_nan[replace_infs_nan != -999]
    
    if not is_high_level:
        array_without_infs_nan = array_without_infs_nan[array_without_infs_nan != -1000]
    
    min_excluding_infs_nan = np.min(array_without_infs_nan)
    if print_outputs:
        print('Min excluding infs/nan: ', min_excluding_infs_nan)
        print()
    if min_excluding_infs_nan == -1.:
        min_without_minus1 = np.min(array_without_infs_nan[array_without_infs_nan != -1.])
        if min_without_minus1 < 0:
            if print_outputs:
                print('Is probably bound by -1 (candidate for default), but please verify.')
            final_min = -1.
        else:
            if print_outputs:
                print('Is probably an integer variable or discriminator or has additional requirements on a secondary vertex!\n\
                    This would be the final minimum (candidate for default), but please verify.')
            final_min = min_without_minus1
    else:
        if print_outputs:
            print('Found a candidate for the default, but please verify.')
        final_min = min_excluding_infs_nan
    if print_outputs:
        print('final_min: ', final_min)
    
    return [original_min, original_max, min_including_infs_nan, min_excluding_infs_nan, final_min]

In [194]:
a1_min = investigate_default(a1, 0, True)

jet_pt

Min:	22.9141		Max:	884.806
Min after replacing infs/nan:  22.9141
Min excluding infs/nan:  22.9141

Found a candidate for the default, but please verify.
final_min:  22.9141


Keep, but probably not necessary at all.

In [195]:
np.save(defaults_path+'calculated_final_minimum_{}.npy'.format('a1'), a1_min[-1])

In [196]:
np.save(defaults_path+'custom_minimum_{}.npy'.format('a1'), a1_min[-1])

This is just to make sure we scale the true minima of the actual distribution and do not ignore them.

In [271]:
np.save(defaults_path+'custom_default_{}.npy'.format('a1'), np.array(0))

In [197]:
a2_min = investigate_default(a2, 1, True)

jet_eta

Min:	-2.5		Max:	2.5
Min after replacing infs/nan:  -2.5
Min excluding infs/nan:  -2.5

Found a candidate for the default, but please verify.
final_min:  -2.5


Keep, but probably not necessary at all.

In [198]:
np.save(defaults_path+'calculated_final_minimum_{}.npy'.format('a2'), a2_min[-1])

In [199]:
np.save(defaults_path+'custom_minimum_{}.npy'.format('a2'), a2_min[-1])

In [272]:
np.save(defaults_path+'custom_default_{}.npy'.format('a2'), np.array(-3))

In [200]:
a3_min = investigate_default(a3, 2, True)

flavor

Min:	0		Max:	5
Min after replacing infs/nan:  0
Min excluding infs/nan:  0

Found a candidate for the default, but please verify.
final_min:  0


Keep, but probably not necessary at all.

In [201]:
np.save(defaults_path+'calculated_final_minimum_{}.npy'.format('a3'), a3_min[-1])

In [202]:
np.save(defaults_path+'custom_minimum_{}.npy'.format('a3'), a3_min[-1])

In [273]:
np.save(defaults_path+'custom_default_{}.npy'.format('a3'), np.array(-1))

In [203]:
a4_0_min = investigate_default(a4[:,0], 3, True)

track_2_d0_significance

Min:	-inf		Max:	273.903
Min after replacing infs/nan:  -999.0
Min excluding infs/nan:  1.03966e-06

Found a candidate for the default, but please verify.
final_min:  1.03966e-06


Zero would be a good option here.

In [204]:
a4_1_min = investigate_default(a4[:,1], 4, True)

track_3_d0_significance

Min:	-inf		Max:	246.409
Min after replacing infs/nan:  -999.0
Min excluding infs/nan:  1.68185e-06

Found a candidate for the default, but please verify.
final_min:  1.68185e-06


Zero would be a good option here.

In [205]:
a4_2_min = investigate_default(a4[:,2], 5, True)

track_2_z0_significance

Min:	-inf		Max:	1863.84
Min after replacing infs/nan:  -999.0
Min excluding infs/nan:  9.09893e-08

Found a candidate for the default, but please verify.
final_min:  9.09893e-08


Zero would be a good option here.

In [206]:
a4_3_min = investigate_default(a4[:,3], 6, True)

track_3_z0_significance

Min:	-inf		Max:	2394.95
Min after replacing infs/nan:  -999.0
Min excluding infs/nan:  3.76541e-08

Found a candidate for the default, but please verify.
final_min:  3.76541e-08


Zero would be a good option here.

In [207]:
a4_4_min = investigate_default(a4[:,4], 7, True)

n_tracks_over_d0_threshold

Min:	0.0		Max:	19.0
Min after replacing infs/nan:  0.0
Min excluding infs/nan:  0.0

Found a candidate for the default, but please verify.
final_min:  0.0


Could set to 0.

In [208]:
a4_5_min = investigate_default(a4[:,5], 8, True)

jet_prob

Min:	-1.0		Max:	0.43385
Min after replacing infs/nan:  -1.0
Min excluding infs/nan:  -1.0

Is probably an integer variable or discriminator or has additional requirements on a secondary vertex!
                    This would be the final minimum (candidate for default), but please verify.
final_min:  0.0


Could set to 0.

In [209]:
a4_6_min = investigate_default(a4[:,6], 9, True)

jet_width_eta

Min:	-inf		Max:	0.39987
Min after replacing infs/nan:  -999.0
Min excluding infs/nan:  9.56622e-09

Found a candidate for the default, but please verify.
final_min:  9.56622e-09


Zero would be a good option here.

In [210]:
a4_7_min = investigate_default(a4[:,7], 10, True)

jet_width_phi

Min:	-inf		Max:	0.399778
Min after replacing infs/nan:  -999.0
Min excluding infs/nan:  1.60691e-07

Found a candidate for the default, but please verify.
final_min:  1.60691e-07


Could set to 0.

In [211]:
np.save(defaults_path+'calculated_final_minima_{}.npy'.format('a4'), np.array([a4_0_min[-1],
                                                                               a4_1_min[-1],
                                                                               a4_2_min[-1],
                                                                               a4_3_min[-1],
                                                                               a4_4_min[-1],
                                                                               a4_5_min[-1],
                                                                               a4_6_min[-1],
                                                                               a4_7_min[-1]]))

Zero would be a good option here.

In [212]:
np.save(defaults_path+'custom_minima_{}.npy'.format('a4'), np.array([0,
                                                                     0,
                                                                     0,
                                                                     0,
                                                                     0,
                                                                     0,
                                                                     0,
                                                                     0]))

To not cause any interference with actual Zeros, slightly shift the numbers.

In [274]:
np.save(defaults_path+'custom_defaults_{}.npy'.format('a4'), np.array([0,
                                                                     0,
                                                                     0,
                                                                     0,
                                                                     -0.001, # 0 would be a problem because there are no defaults, all 0s are true 0s
                                                                     -0.001, # 0 would be a problem because there are true 0s, and some defaults
                                                                     0,
                                                                     0]))

In [213]:
a5_0_min = investigate_default(a5[:,0], 11, True)

vertex_significance

Min:	nan		Max:	nan
Min after replacing infs/nan:  -999.0
Min excluding infs/nan:  -1.0

Is probably an integer variable or discriminator or has additional requirements on a secondary vertex!
                    This would be the final minimum (candidate for default), but please verify.
final_min:  0.00446355


Zero would be a good option here.

In [214]:
a5_1_min = investigate_default(a5[:,1], 12, True)

n_secondary_vertices

Min:	-1.0		Max:	12.0
Min after replacing infs/nan:  -1.0
Min excluding infs/nan:  -1.0

Is probably an integer variable or discriminator or has additional requirements on a secondary vertex!
                    This would be the final minimum (candidate for default), but please verify.
final_min:  1.0


Could set to 0.

In [215]:
a5_2_min = investigate_default(a5[:,2], 13, True)

n_secondary_vertex_tracks

Min:	-1.0		Max:	52.0
Min after replacing infs/nan:  -1.0
Min excluding infs/nan:  -1.0

Is probably an integer variable or discriminator or has additional requirements on a secondary vertex!
                    This would be the final minimum (candidate for default), but please verify.
final_min:  2.0


Could set to 0.

In [216]:
a5_3_min = investigate_default(a5[:,3], 14, True)

delta_r_vertex

Min:	6.52356e-05		Max:	inf
Min after replacing infs/nan:  -999.0
Min excluding infs/nan:  6.52356e-05

Found a candidate for the default, but please verify.
final_min:  6.52356e-05


Zero would be a good option here.

In [217]:
a5_4_min = investigate_default(a5[:,4], 15, True)

vertex_mass

Min:	-1.0		Max:	25.8516
Min after replacing infs/nan:  -1.0
Min excluding infs/nan:  -1.0

Is probably an integer variable or discriminator or has additional requirements on a secondary vertex!
                    This would be the final minimum (candidate for default), but please verify.
final_min:  0.279139


Zero would be a good option here.

In [218]:
a5_5_min = investigate_default(a5[:,5], 16, True)

vertex_energy_fraction

Min:	-inf		Max:	4.7216
Min after replacing infs/nan:  -999.0
Min excluding infs/nan:  0.00912945

Found a candidate for the default, but please verify.
final_min:  0.00912945


Could set to 0.

In [219]:
np.save(defaults_path+'calculated_final_minima_{}.npy'.format('a5'), np.array([a5_0_min[-1],
                                                                               a5_1_min[-1],
                                                                               a5_2_min[-1],
                                                                               a5_3_min[-1],
                                                                               a5_4_min[-1],
                                                                               a5_5_min[-1]]))

Zero would be a good option here.

In [220]:
np.save(defaults_path+'custom_minima_{}.npy'.format('a5'), np.array([0,
                                                                     0,
                                                                     0,
                                                                     0,
                                                                     0,
                                                                     0]))

There are no actual exact Zeros in the dataset, so setting to 0 remains safe.

In [275]:
np.save(defaults_path+'custom_defaults_{}.npy'.format('a5'), np.array([0,
                                                                     0,
                                                                     0,
                                                                     0,
                                                                     0,
                                                                     0]))

## Investigate low-level variables

Select a feature and check defaults for up to 33 tracks in the jet.

Prototype ideas:

In [82]:
var_names_track_and_vertex[0]

'D0'

In [83]:
var_names_track_and_vertex[1]

'Z0'

In [85]:
def get_track_vertex_indices_all_tracks_one_variable(index):
    return [index * 33 + k for k in range(33)]

In [88]:
vars_1 = get_track_vertex_indices_all_tracks_one_variable(1)

In [110]:
def get_low_level_defaults_info(var_index):
    vars_ = get_track_vertex_indices_all_tracks_one_variable(var_index)
    columns = np.concatenate([np.load(np_arrays_path+'sorted_track_vertex_info_split_{0}_{1}.npy'.format(splits[i],splits[i+1]-1))[:,vars_] for i in range(len(splits)-1)])

    
    final_minima = []
    
    for k in range(33):
        var_k = vars_[k]
        index = var_k + n_high_level
        info_default_options = investigate_default(columns[:,k], index, is_high_level=False, print_outputs=False)
        print(info_default_options)
        final_minima.append(info_default_options[-1])
        print()
        print()
        
    np.save(defaults_path+'calculated_final_minima_track_vertex_var_{}.npy'.format(var_index), np.array(final_minima))
    
    return final_minima

In [111]:
all_track_vertex_defaults = []

In [112]:
all_track_vertex_defaults.append(get_low_level_defaults_info(0))

track_0_D0

[-7.92794, 7.99996, -7.92794, -7.92794, -7.92794]


track_1_D0

[-1000.0, 7.97893, -1000.0, -7.99466, -7.99466]


track_2_D0

[-1000.0, 4.26448, -1000.0, -7.99902, -7.99902]


track_3_D0

[-1000.0, 2.69328, -1000.0, -7.99968, -7.99968]


track_4_D0

[-1000.0, 1.83259, -1000.0, -7.99789, -7.99789]


track_5_D0

[-1000.0, 1.33952, -1000.0, -7.99928, -7.99928]


track_6_D0

[-1000.0, 1.10826, -1000.0, -7.99755, -7.99755]


track_7_D0

[-1000.0, 0.953104, -1000.0, -7.99724, -7.99724]


track_8_D0

[-1000.0, 0.388905, -1000.0, -7.99764, -7.99764]


track_9_D0

[-1000.0, 0.119298, -1000.0, -7.98958, -7.98958]


track_10_D0

[-1000.0, 0.0702186, -1000.0, -7.99895, -7.99895]


track_11_D0

[-1000.0, 0.037299, -1000.0, -7.99545, -7.99545]


track_12_D0

[-1000.0, 0.0292707, -1000.0, -7.96536, -7.96536]


track_13_D0

[-1000.0, 0.0225356, -1000.0, -7.95577, -7.95577]


track_14_D0

[-1000.0, 0.0139734, -1000.0, -7.82211, -7.82211]


track_15_D0

[-1000.0, 0.0118958, -1000.0, -7.52736

Keep as they are.

In [156]:
len(all_track_vertex_defaults)

28

In [157]:
feature_index_track_vertex = 0
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])

Do not use the exact minima, but slightly shift them to not cause interference with true values.

In [277]:
feature_index_track_vertex = 0
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.array(all_track_vertex_defaults[feature_index_track_vertex])-0.001)

In [113]:
all_track_vertex_defaults.append(get_low_level_defaults_info(1))

track_0_Z0

[-393.904, 320.044, -393.904, -393.904, -393.904]


track_1_Z0

[-1000.0, 185.743, -1000.0, -185.665, -185.665]


track_2_Z0

[-1000.0, 228.096, -1000.0, -253.083, -253.083]


track_3_Z0

[-1000.0, 201.736, -1000.0, -237.928, -237.928]


track_4_Z0

[-1000.0, 338.585, -1000.0, -209.215, -209.215]


track_5_Z0

[-1000.0, 170.146, -1000.0, -272.432, -272.432]


track_6_Z0

[-1000.0, 184.23, -1000.0, -214.421, -214.421]


track_7_Z0

[-1000.0, 276.554, -1000.0, -282.324, -282.324]


track_8_Z0

[-1000.0, 98.4339, -1000.0, -108.551, -108.551]


track_9_Z0

[-1000.0, 123.901, -1000.0, -184.0, -184.0]


track_10_Z0

[-1000.0, 41.8285, -1000.0, -238.852, -238.852]


track_11_Z0

[-1000.0, 47.4188, -1000.0, -119.193, -119.193]


track_12_Z0

[-1000.0, 49.6708, -1000.0, -64.4819, -64.4819]


track_13_Z0

[-1000.0, 34.3598, -1000.0, -43.7101, -43.7101]


track_14_Z0

[-1000.0, 26.6612, -1000.0, -37.5456, -37.5456]


track_15_Z0

[-1000.0, 19.5342, -1000.0, -39.913, -39.913]


track_1

Keep as they are (for now, but some values might be a bit high).

In [158]:
feature_index_track_vertex = 1
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])

In [278]:
feature_index_track_vertex = 1
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.array(all_track_vertex_defaults[feature_index_track_vertex])-0.001)

In [114]:
all_track_vertex_defaults.append(get_low_level_defaults_info(2))

track_0_PHI

[-0.399934, 0.399982, -0.399934, -0.399934, -0.399934]


track_1_PHI

[-1000.0, 0.399973, -1000.0, -0.399985, -0.399985]


track_2_PHI

[-1000.0, 0.399983, -1000.0, -0.399956, -0.399956]


track_3_PHI

[-1000.0, 0.39995, -1000.0, -0.399959, -0.399959]


track_4_PHI

[-1000.0, 0.399933, -1000.0, -0.399948, -0.399948]


track_5_PHI

[-1000.0, 0.399902, -1000.0, -0.39993, -0.39993]


track_6_PHI

[-1000.0, 0.399881, -1000.0, -0.399828, -0.399828]


track_7_PHI

[-1000.0, 0.399428, -1000.0, -0.399824, -0.399824]


track_8_PHI

[-1000.0, 0.399857, -1000.0, -0.399913, -0.399913]


track_9_PHI

[-1000.0, 0.399905, -1000.0, -0.399926, -0.399926]


track_10_PHI

[-1000.0, 0.398192, -1000.0, -0.399541, -0.399541]


track_11_PHI

[-1000.0, 0.399018, -1000.0, -0.399291, -0.399291]


track_12_PHI

[-1000.0, 0.396921, -1000.0, -0.399238, -0.399238]


track_13_PHI

[-1000.0, 0.393594, -1000.0, -0.396917, -0.396917]


track_14_PHI

[-1000.0, 0.39472, -1000.0, -0.398698, -0.398698]


track

Keep as they are.

In [159]:
feature_index_track_vertex = 2
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])

In [279]:
feature_index_track_vertex = 2
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.array(all_track_vertex_defaults[feature_index_track_vertex])-0.001)

In [115]:
all_track_vertex_defaults.append(get_low_level_defaults_info(3))

track_0_THETA

[-0.396828, 0.39653, -0.396828, -0.396828, -0.396828]


track_1_THETA

[-1000.0, 0.397203, -1000.0, -0.397026, -0.397026]


track_2_THETA

[-1000.0, 0.396986, -1000.0, -0.396841, -0.396841]


track_3_THETA

[-1000.0, 0.396983, -1000.0, -0.396209, -0.396209]


track_4_THETA

[-1000.0, 0.396764, -1000.0, -0.39652, -0.39652]


track_5_THETA

[-1000.0, 0.396997, -1000.0, -0.395707, -0.395707]


track_6_THETA

[-1000.0, 0.395814, -1000.0, -0.39472, -0.39472]


track_7_THETA

[-1000.0, 0.395428, -1000.0, -0.395413, -0.395413]


track_8_THETA

[-1000.0, 0.394757, -1000.0, -0.395253, -0.395253]


track_9_THETA

[-1000.0, 0.393269, -1000.0, -0.394414, -0.394414]


track_10_THETA

[-1000.0, 0.39351, -1000.0, -0.393643, -0.393643]


track_11_THETA

[-1000.0, 0.396207, -1000.0, -0.390349, -0.390349]


track_12_THETA

[-1000.0, 0.387266, -1000.0, -0.392818, -0.392818]


track_13_THETA

[-1000.0, 0.386793, -1000.0, -0.385795, -0.385795]


track_14_THETA

[-1000.0, 0.389358, -1000.0, -

Keep as they are.

In [160]:
feature_index_track_vertex = 3
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])

In [280]:
feature_index_track_vertex = 3
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.array(all_track_vertex_defaults[feature_index_track_vertex])-0.001)

In [116]:
all_track_vertex_defaults.append(get_low_level_defaults_info(4))

track_0_QOVERP

[-1.99989, 1.99933, -1.99989, -1.99989, -1.99989]


track_1_QOVERP

[-1000.0, 1.99929, -1000.0, -1.99989, -1.99989]


track_2_QOVERP

[-1000.0, 1.99966, -1000.0, -1.99961, -1.99961]


track_3_QOVERP

[-1000.0, 1.99924, -1000.0, -1.99939, -1.99939]


track_4_QOVERP

[-1000.0, 1.9993, -1000.0, -1.99967, -1.99967]


track_5_QOVERP

[-1000.0, 1.99936, -1000.0, -1.99767, -1.99767]


track_6_QOVERP

[-1000.0, 1.99964, -1000.0, -1.99906, -1.99906]


track_7_QOVERP

[-1000.0, 1.99605, -1000.0, -1.99719, -1.99719]


track_8_QOVERP

[-1000.0, 1.99782, -1000.0, -1.99839, -1.99839]


track_9_QOVERP

[-1000.0, 1.99749, -1000.0, -1.99613, -1.99613]


track_10_QOVERP

[-1000.0, 1.99287, -1000.0, -1.99038, -1.99038]


track_11_QOVERP

[-1000.0, 1.98229, -1000.0, -1.98943, -1.98943]


track_12_QOVERP

[-1000.0, 1.98385, -1000.0, -1.99127, -1.99127]


track_13_QOVERP

[-1000.0, 1.96945, -1000.0, -1.94737, -1.94737]


track_14_QOVERP

[-1000.0, 1.97599, -1000.0, -1.94361, -1.94361]


trac

Keep as they are (though -2 seems like a good idea for the future).

In [161]:
feature_index_track_vertex = 4
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])

In [281]:
feature_index_track_vertex = 4
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.array(all_track_vertex_defaults[feature_index_track_vertex])-0.001)

In [117]:
all_track_vertex_defaults.append(get_low_level_defaults_info(5))

track_0_D0D0

[1.62516e-05, 0.00245914, 1.62516e-05, 1.62516e-05, 1.62516e-05]


track_1_D0D0

[-1000.0, 0.00245914, -1000.0, 1.62516e-05, 1.62516e-05]


track_2_D0D0

[-1000.0, 0.00245914, -1000.0, 1.62516e-05, 1.62516e-05]


track_3_D0D0

[-1000.0, 0.00245914, -1000.0, 1.62516e-05, 1.62516e-05]


track_4_D0D0

[-1000.0, 0.00245914, -1000.0, 1.92518e-05, 1.92518e-05]


track_5_D0D0

[-1000.0, 0.00245914, -1000.0, 1.98072e-05, 1.98072e-05]


track_6_D0D0

[-1000.0, 0.00245914, -1000.0, 1.79211e-05, 1.79211e-05]


track_7_D0D0

[-1000.0, 0.00245914, -1000.0, 1.98072e-05, 1.98072e-05]


track_8_D0D0

[-1000.0, 0.00245914, -1000.0, 1.98072e-05, 1.98072e-05]


track_9_D0D0

[-1000.0, 0.00245914, -1000.0, 1.98072e-05, 1.98072e-05]


track_10_D0D0

[-1000.0, 0.00245914, -1000.0, 2.29964e-05, 2.29964e-05]


track_11_D0D0

[-1000.0, 0.00245914, -1000.0, 2.29964e-05, 2.29964e-05]


track_12_D0D0

[-1000.0, 0.00245914, -1000.0, 2.14414e-05, 2.14414e-05]


track_13_D0D0

[-1000.0, 0.00245914, -10

0 would be good for all.

In [162]:
feature_index_track_vertex = 5
#np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [282]:
feature_index_track_vertex = 5
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [118]:
all_track_vertex_defaults.append(get_low_level_defaults_info(6))

track_0_Z0D0

[-3.45404e-05, 0.000703694, -3.45404e-05, -3.45404e-05, -3.45404e-05]


track_1_Z0D0

[-1000.0, 0.000703694, -1000.0, -3.45404e-05, -3.45404e-05]


track_2_Z0D0

[-1000.0, 0.000703694, -1000.0, -3.45404e-05, -3.45404e-05]


track_3_Z0D0

[-1000.0, 0.000703694, -1000.0, -3.45404e-05, -3.45404e-05]


track_4_Z0D0

[-1000.0, 0.000703694, -1000.0, -3.45404e-05, -3.45404e-05]


track_5_Z0D0

[-1000.0, 0.000703694, -1000.0, -3.45404e-05, -3.45404e-05]


track_6_Z0D0

[-1000.0, 0.000703694, -1000.0, -3.45404e-05, -3.45404e-05]


track_7_Z0D0

[-1000.0, 0.000703694, -1000.0, -3.45404e-05, -3.45404e-05]


track_8_Z0D0

[-1000.0, 0.000703694, -1000.0, -3.45404e-05, -3.45404e-05]


track_9_Z0D0

[-1000.0, 0.000703694, -1000.0, -3.45404e-05, -3.45404e-05]


track_10_Z0D0

[-1000.0, 0.000703694, -1000.0, -3.45404e-05, -3.45404e-05]


track_11_Z0D0

[-1000.0, 0.000703694, -1000.0, -3.45404e-05, -3.45404e-05]


track_12_Z0D0

[-1000.0, 0.000703694, -1000.0, -3.45404e-05, -3.45404e-05]



I would say keep, negative values means that 0 is not a viable option here (convariance can be negative for non-diagonal elements I guess).

In [163]:
feature_index_track_vertex = 6
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])
#np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [324]:
feature_index_track_vertex = 6
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.array(all_track_vertex_defaults[feature_index_track_vertex])-1e-6)

In [119]:
all_track_vertex_defaults.append(get_low_level_defaults_info(7))

track_0_Z0Z0

[0.00163113, 0.221921, 0.00163113, 0.00163113, 0.00163113]


track_1_Z0Z0

[-1000.0, 0.221921, -1000.0, 0.00163113, 0.00163113]


track_2_Z0Z0

[-1000.0, 0.221921, -1000.0, 0.00163113, 0.00163113]


track_3_Z0Z0

[-1000.0, 0.221921, -1000.0, 0.00163113, 0.00163113]


track_4_Z0Z0

[-1000.0, 0.221921, -1000.0, 0.00163113, 0.00163113]


track_5_Z0Z0

[-1000.0, 0.221921, -1000.0, 0.00163113, 0.00163113]


track_6_Z0Z0

[-1000.0, 0.221921, -1000.0, 0.001534, 0.001534]


track_7_Z0Z0

[-1000.0, 0.221921, -1000.0, 0.00163113, 0.00163113]


track_8_Z0Z0

[-1000.0, 0.221921, -1000.0, 0.00163113, 0.00163113]


track_9_Z0Z0

[-1000.0, 0.221921, -1000.0, 0.00163113, 0.00163113]


track_10_Z0Z0

[-1000.0, 0.221921, -1000.0, 0.00163113, 0.00163113]


track_11_Z0Z0

[-1000.0, 0.221921, -1000.0, 0.00163113, 0.00163113]


track_12_Z0Z0

[-1000.0, 0.221921, -1000.0, 0.00163113, 0.00163113]


track_13_Z0Z0

[-1000.0, 0.221921, -1000.0, 0.00163113, 0.00163113]


track_14_Z0Z0

[-1000.0, 0.2

Diagonal element --> 0 would be good for all.

In [164]:
feature_index_track_vertex = 7
#np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [284]:
feature_index_track_vertex = 7
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [120]:
all_track_vertex_defaults.append(get_low_level_defaults_info(8))

track_0_PHID0

[-2.12598e-05, -1.44159e-07, -2.12598e-05, -2.12598e-05, -2.12598e-05]


track_1_PHID0

[-1000.0, -1.44159e-07, -1000.0, -2.12598e-05, -2.12598e-05]


track_2_PHID0

[-1000.0, -1.44159e-07, -1000.0, -2.12598e-05, -2.12598e-05]


track_3_PHID0

[-1000.0, -1.44159e-07, -1000.0, -2.12598e-05, -2.12598e-05]


track_4_PHID0

[-1000.0, -1.89077e-07, -1000.0, -2.12598e-05, -2.12598e-05]


track_5_PHID0

[-1000.0, -2.04204e-07, -1000.0, -2.12598e-05, -2.12598e-05]


track_6_PHID0

[-1000.0, -1.73618e-07, -1000.0, -2.12598e-05, -2.12598e-05]


track_7_PHID0

[-1000.0, -2.04204e-07, -1000.0, -2.12598e-05, -2.12598e-05]


track_8_PHID0

[-1000.0, -2.04204e-07, -1000.0, -2.12598e-05, -2.12598e-05]


track_9_PHID0

[-1000.0, -2.04204e-07, -1000.0, -2.12598e-05, -2.12598e-05]


track_10_PHID0

[-1000.0, -2.30439e-07, -1000.0, -2.12598e-05, -2.12598e-05]


track_11_PHID0

[-1000.0, -2.30439e-07, -1000.0, -2.12598e-05, -2.12598e-05]


track_12_PHID0

[-1000.0, -2.10801e-07, -1000.0, -2.

I would say keep, negative values means that 0 is not a viable option here (convariance can be negative for non-diagonal elements I guess).

In [165]:
feature_index_track_vertex = 8
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])
#np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [325]:
feature_index_track_vertex = 8
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.array(all_track_vertex_defaults[feature_index_track_vertex])-1e-6)

In [121]:
all_track_vertex_defaults.append(get_low_level_defaults_info(9))

track_0_PHIZ0

[-2.47313e-06, 2.9672e-07, -2.47313e-06, -2.47313e-06, -2.47313e-06]


track_1_PHIZ0

[-1000.0, 2.9672e-07, -1000.0, -2.47313e-06, -2.47313e-06]


track_2_PHIZ0

[-1000.0, 3.02194e-07, -1000.0, -2.47313e-06, -2.47313e-06]


track_3_PHIZ0

[-1000.0, 2.9672e-07, -1000.0, -2.47313e-06, -2.47313e-06]


track_4_PHIZ0

[-1000.0, 2.2186e-07, -1000.0, -2.47313e-06, -2.47313e-06]


track_5_PHIZ0

[-1000.0, 2.2186e-07, -1000.0, -2.47313e-06, -2.47313e-06]


track_6_PHIZ0

[-1000.0, 2.2186e-07, -1000.0, -2.47313e-06, -2.47313e-06]


track_7_PHIZ0

[-1000.0, 2.2186e-07, -1000.0, -2.47313e-06, -2.47313e-06]


track_8_PHIZ0

[-1000.0, 2.2186e-07, -1000.0, -2.47313e-06, -2.47313e-06]


track_9_PHIZ0

[-1000.0, 2.2186e-07, -1000.0, -2.47313e-06, -2.47313e-06]


track_10_PHIZ0

[-1000.0, 2.2186e-07, -1000.0, -2.47313e-06, -2.47313e-06]


track_11_PHIZ0

[-1000.0, 2.2186e-07, -1000.0, -2.47313e-06, -2.47313e-06]


track_12_PHIZ0

[-1000.0, 2.2186e-07, -1000.0, -2.47313e-06, -2.47313e-06]


I would say keep, negative values means that 0 is not a viable option here (convariance can be negative for non-diagonal elements I guess).

In [166]:
feature_index_track_vertex = 9
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])
#np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [326]:
feature_index_track_vertex = 9
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.array(all_track_vertex_defaults[feature_index_track_vertex])-1e-7)

In [122]:
all_track_vertex_defaults.append(get_low_level_defaults_info(10))

track_0_PHIPHI

[4.18109e-09, 2.41633e-07, 4.18109e-09, 4.18109e-09, 4.18109e-09]


track_1_PHIPHI

[-1000.0, 2.41633e-07, -1000.0, 4.18109e-09, 4.18109e-09]


track_2_PHIPHI

[-1000.0, 2.41633e-07, -1000.0, 4.18109e-09, 4.18109e-09]


track_3_PHIPHI

[-1000.0, 2.41633e-07, -1000.0, 4.18109e-09, 4.18109e-09]


track_4_PHIPHI

[-1000.0, 2.41633e-07, -1000.0, 5.24128e-09, 5.24128e-09]


track_5_PHIPHI

[-1000.0, 2.41633e-07, -1000.0, 5.69284e-09, 5.69284e-09]


track_6_PHIPHI

[-1000.0, 2.41633e-07, -1000.0, 4.21592e-09, 4.21592e-09]


track_7_PHIPHI

[-1000.0, 2.41633e-07, -1000.0, 5.87261e-09, 5.87261e-09]


track_8_PHIPHI

[-1000.0, 2.41633e-07, -1000.0, 5.87261e-09, 5.87261e-09]


track_9_PHIPHI

[-1000.0, 2.41633e-07, -1000.0, 5.87261e-09, 5.87261e-09]


track_10_PHIPHI

[-1000.0, 2.41633e-07, -1000.0, 7.33958e-09, 7.33958e-09]


track_11_PHIPHI

[-1000.0, 2.41633e-07, -1000.0, 7.33958e-09, 7.33958e-09]


track_12_PHIPHI

[-1000.0, 2.41633e-07, -1000.0, 5.69284e-09, 5.69284e-09]


t

Diagonal, use 0.

In [167]:
feature_index_track_vertex = 10
#np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [287]:
feature_index_track_vertex = 10
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [123]:
all_track_vertex_defaults.append(get_low_level_defaults_info(11))

track_0_THETAD0

[-1.53849e-07, 1.32982e-07, -1.53849e-07, -1.53849e-07, -1.53849e-07]


track_1_THETAD0

[-1000.0, 1.32982e-07, -1000.0, -1.53849e-07, -1.53849e-07]


track_2_THETAD0

[-1000.0, 1.32982e-07, -1000.0, -1.53849e-07, -1.53849e-07]


track_3_THETAD0

[-1000.0, 1.32982e-07, -1000.0, -1.53849e-07, -1.53849e-07]


track_4_THETAD0

[-1000.0, 1.32982e-07, -1000.0, -1.53849e-07, -1.53849e-07]


track_5_THETAD0

[-1000.0, 8.83371e-08, -1000.0, -1.53849e-07, -1.53849e-07]


track_6_THETAD0

[-1000.0, 1.32982e-07, -1000.0, -1.53849e-07, -1.53849e-07]


track_7_THETAD0

[-1000.0, 8.83371e-08, -1000.0, -1.53849e-07, -1.53849e-07]


track_8_THETAD0

[-1000.0, 1.32982e-07, -1000.0, -1.53849e-07, -1.53849e-07]


track_9_THETAD0

[-1000.0, 1.32982e-07, -1000.0, -1.53849e-07, -1.53849e-07]


track_10_THETAD0

[-1000.0, 8.83371e-08, -1000.0, -1.53849e-07, -1.53849e-07]


track_11_THETAD0

[-1000.0, 8.83371e-08, -1000.0, -1.53849e-07, -1.53849e-07]


track_12_THETAD0

[-1000.0, 8.83371e-08,

I would say keep, negative values means that 0 is not a viable option here (convariance can be negative for non-diagonal elements I guess).

In [168]:
feature_index_track_vertex = 11
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])
#np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [327]:
feature_index_track_vertex = 11
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.array(all_track_vertex_defaults[feature_index_track_vertex])-1e-8)

In [124]:
all_track_vertex_defaults.append(get_low_level_defaults_info(12))

track_0_THETAZ0

[2.3957e-06, 7.87844e-05, 2.3957e-06, 2.3957e-06, 2.3957e-06]


track_1_THETAZ0

[-1000.0, 7.87844e-05, -1000.0, 2.30425e-06, 2.30425e-06]


track_2_THETAZ0

[-1000.0, 7.87844e-05, -1000.0, 1.81567e-06, 1.81567e-06]


track_3_THETAZ0

[-1000.0, 7.87844e-05, -1000.0, 2.42289e-06, 2.42289e-06]


track_4_THETAZ0

[-1000.0, 7.87844e-05, -1000.0, 2.42289e-06, 2.42289e-06]


track_5_THETAZ0

[-1000.0, 7.87844e-05, -1000.0, 2.42289e-06, 2.42289e-06]


track_6_THETAZ0

[-1000.0, 7.87844e-05, -1000.0, 2.42289e-06, 2.42289e-06]


track_7_THETAZ0

[-1000.0, 7.87844e-05, -1000.0, 2.42289e-06, 2.42289e-06]


track_8_THETAZ0

[-1000.0, 7.87844e-05, -1000.0, 2.42289e-06, 2.42289e-06]


track_9_THETAZ0

[-1000.0, 7.87844e-05, -1000.0, 2.42289e-06, 2.42289e-06]


track_10_THETAZ0

[-1000.0, 7.87844e-05, -1000.0, 2.42289e-06, 2.42289e-06]


track_11_THETAZ0

[-1000.0, 7.87844e-05, -1000.0, 2.42289e-06, 2.42289e-06]


track_12_THETAZ0

[-1000.0, 7.87844e-05, -1000.0, 2.42289e-06, 2.42289

Keep, not diagonal, although all are positive and small.

In [169]:
feature_index_track_vertex = 12
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])
#np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [328]:
feature_index_track_vertex = 12
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.array(all_track_vertex_defaults[feature_index_track_vertex])-1e-6)

In [125]:
all_track_vertex_defaults.append(get_low_level_defaults_info(13))

track_0_THETAPHI

[-1.38111e-09, 2.03563e-09, -1.38111e-09, -1.38111e-09, -1.38111e-09]


track_1_THETAPHI

[-1000.0, 2.03563e-09, -1000.0, -1.17775e-09, -1.17775e-09]


track_2_THETAPHI

[-1000.0, 2.03563e-09, -1000.0, -1.38111e-09, -1.38111e-09]


track_3_THETAPHI

[-1000.0, 2.03563e-09, -1000.0, -1.17775e-09, -1.17775e-09]


track_4_THETAPHI

[-1000.0, 1.22249e-09, -1000.0, -1.17775e-09, -1.17775e-09]


track_5_THETAPHI

[-1000.0, 1.22249e-09, -1000.0, -1.17775e-09, -1.17775e-09]


track_6_THETAPHI

[-1000.0, 1.22249e-09, -1000.0, -1.17775e-09, -1.17775e-09]


track_7_THETAPHI

[-1000.0, 1.22249e-09, -1000.0, -1.17775e-09, -1.17775e-09]


track_8_THETAPHI

[-1000.0, 1.22249e-09, -1000.0, -1.17775e-09, -1.17775e-09]


track_9_THETAPHI

[-1000.0, 1.22249e-09, -1000.0, -1.17775e-09, -1.17775e-09]


track_10_THETAPHI

[-1000.0, 1.22249e-09, -1000.0, -1.17775e-09, -1.17775e-09]


track_11_THETAPHI

[-1000.0, 1.22249e-09, -1000.0, -1.17775e-09, -1.17775e-09]


track_12_THETAPHI

[-1000.0,

I would say keep, negative values means that 0 is not a viable option here (convariance can be negative for non-diagonal elements I guess).

In [170]:
feature_index_track_vertex = 13
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])
#np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [329]:
feature_index_track_vertex = 13
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.array(all_track_vertex_defaults[feature_index_track_vertex])-1e-10)

In [126]:
all_track_vertex_defaults.append(get_low_level_defaults_info(14))

track_0_THETATHETA

[4.9851e-09, 4.80983e-07, 4.9851e-09, 4.9851e-09, 4.9851e-09]


track_1_THETATHETA

[-1000.0, 4.80983e-07, -1000.0, 4.9851e-09, 4.9851e-09]


track_2_THETATHETA

[-1000.0, 4.80983e-07, -1000.0, 4.81434e-09, 4.81434e-09]


track_3_THETATHETA

[-1000.0, 4.80983e-07, -1000.0, 4.9851e-09, 4.9851e-09]


track_4_THETATHETA

[-1000.0, 4.80983e-07, -1000.0, 4.9851e-09, 4.9851e-09]


track_5_THETATHETA

[-1000.0, 4.80983e-07, -1000.0, 4.9851e-09, 4.9851e-09]


track_6_THETATHETA

[-1000.0, 4.80983e-07, -1000.0, 4.9851e-09, 4.9851e-09]


track_7_THETATHETA

[-1000.0, 4.80983e-07, -1000.0, 4.9851e-09, 4.9851e-09]


track_8_THETATHETA

[-1000.0, 4.80983e-07, -1000.0, 5.91178e-09, 5.91178e-09]


track_9_THETATHETA

[-1000.0, 4.80983e-07, -1000.0, 4.9851e-09, 4.9851e-09]


track_10_THETATHETA

[-1000.0, 4.80983e-07, -1000.0, 5.96309e-09, 5.96309e-09]


track_11_THETATHETA

[-1000.0, 4.80983e-07, -1000.0, 5.96309e-09, 5.96309e-09]


track_12_THETATHETA

[-1000.0, 4.80983e-07, -100

Keep as they are (though 0 would be good for all).

In [292]:
feature_index_track_vertex = 14
#np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [291]:
feature_index_track_vertex = 14
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [127]:
all_track_vertex_defaults.append(get_low_level_defaults_info(15))

track_0_QOVERPD0

[-1.77058e-05, -4.12594e-07, -1.77058e-05, -1.77058e-05, -1.77058e-05]


track_1_QOVERPD0

[-1000.0, -4.12594e-07, -1000.0, -1.77058e-05, -1.77058e-05]


track_2_QOVERPD0

[-1000.0, -2.53089e-07, -1000.0, -1.77058e-05, -1.77058e-05]


track_3_QOVERPD0

[-1000.0, -4.12594e-07, -1000.0, -1.77058e-05, -1.77058e-05]


track_4_QOVERPD0

[-1000.0, -4.14285e-07, -1000.0, -1.77058e-05, -1.77058e-05]


track_5_QOVERPD0

[-1000.0, -4.12594e-07, -1000.0, -1.77058e-05, -1.77058e-05]


track_6_QOVERPD0

[-1000.0, -3.77506e-07, -1000.0, -1.77058e-05, -1.77058e-05]


track_7_QOVERPD0

[-1000.0, -4.14285e-07, -1000.0, -1.77058e-05, -1.77058e-05]


track_8_QOVERPD0

[-1000.0, -4.14285e-07, -1000.0, -1.77058e-05, -1.77058e-05]


track_9_QOVERPD0

[-1000.0, -4.14285e-07, -1000.0, -1.77058e-05, -1.77058e-05]


track_10_QOVERPD0

[-1000.0, -4.14285e-07, -1000.0, -1.77058e-05, -1.77058e-05]


track_11_QOVERPD0

[-1000.0, -4.81048e-07, -1000.0, -1.77058e-05, -1.77058e-05]


track_12_QOVERPD

I would say keep, negative values means that 0 is not a viable option here (convariance can be negative for non-diagonal elements I guess).

In [171]:
feature_index_track_vertex = 15
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])
#np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [330]:
feature_index_track_vertex = 15
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.array(all_track_vertex_defaults[feature_index_track_vertex])-1e-6)

In [128]:
all_track_vertex_defaults.append(get_low_level_defaults_info(16))

track_0_QOVERPZ0

[-6.38033e-06, 3.51726e-07, -6.38033e-06, -6.38033e-06, -6.38033e-06]


track_1_QOVERPZ0

[-1000.0, 3.51726e-07, -1000.0, -6.38033e-06, -6.38033e-06]


track_2_QOVERPZ0

[-1000.0, 6.30697e-07, -1000.0, -6.38033e-06, -6.38033e-06]


track_3_QOVERPZ0

[-1000.0, 3.51726e-07, -1000.0, -6.38033e-06, -6.38033e-06]


track_4_QOVERPZ0

[-1000.0, 3.51726e-07, -1000.0, -6.38033e-06, -6.38033e-06]


track_5_QOVERPZ0

[-1000.0, 3.51726e-07, -1000.0, -6.38033e-06, -6.38033e-06]


track_6_QOVERPZ0

[-1000.0, 3.51726e-07, -1000.0, -6.38033e-06, -6.38033e-06]


track_7_QOVERPZ0

[-1000.0, 3.51726e-07, -1000.0, -6.38033e-06, -6.38033e-06]


track_8_QOVERPZ0

[-1000.0, 3.51726e-07, -1000.0, -6.38033e-06, -6.38033e-06]


track_9_QOVERPZ0

[-1000.0, 3.51726e-07, -1000.0, -6.38033e-06, -6.38033e-06]


track_10_QOVERPZ0

[-1000.0, 3.51726e-07, -1000.0, -6.38033e-06, -6.38033e-06]


track_11_QOVERPZ0

[-1000.0, 3.51726e-07, -1000.0, -6.38033e-06, -6.38033e-06]


track_12_QOVERPZ0

[-1000.0,

I would say keep, negative values means that 0 is not a viable option here (convariance can be negative for non-diagonal elements I guess).

In [172]:
feature_index_track_vertex = 16
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])
#np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [331]:
feature_index_track_vertex = 16
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.array(all_track_vertex_defaults[feature_index_track_vertex])-1e-7)

In [129]:
all_track_vertex_defaults.append(get_low_level_defaults_info(17))

track_0_QOVERPPHI

[9.35556e-09, 2.6567e-07, 9.35556e-09, 9.35556e-09, 9.35556e-09]


track_1_QOVERPPHI

[-1000.0, 2.6567e-07, -1000.0, 1.05057e-08, 1.05057e-08]


track_2_QOVERPPHI

[-1000.0, 2.6567e-07, -1000.0, 9.22917e-09, 9.22917e-09]


track_3_QOVERPPHI

[-1000.0, 2.6567e-07, -1000.0, 1.18042e-08, 1.18042e-08]


track_4_QOVERPPHI

[-1000.0, 2.6567e-07, -1000.0, 1.18042e-08, 1.18042e-08]


track_5_QOVERPPHI

[-1000.0, 2.6567e-07, -1000.0, 1.18042e-08, 1.18042e-08]


track_6_QOVERPPHI

[-1000.0, 2.6567e-07, -1000.0, 1.01097e-08, 1.01097e-08]


track_7_QOVERPPHI

[-1000.0, 2.6567e-07, -1000.0, 1.18042e-08, 1.18042e-08]


track_8_QOVERPPHI

[-1000.0, 2.6567e-07, -1000.0, 1.18042e-08, 1.18042e-08]


track_9_QOVERPPHI

[-1000.0, 2.6567e-07, -1000.0, 1.18042e-08, 1.18042e-08]


track_10_QOVERPPHI

[-1000.0, 2.6567e-07, -1000.0, 1.18042e-08, 1.18042e-08]


track_11_QOVERPPHI

[-1000.0, 2.6567e-07, -1000.0, 1.589e-08, 1.589e-08]


track_12_QOVERPPHI

[-1000.0, 2.6567e-07, -1000.0, 1.1913e

Is not diagonal, keeping does not hurt.

In [173]:
feature_index_track_vertex = 17
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])
#np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [332]:
feature_index_track_vertex = 17
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.array(all_track_vertex_defaults[feature_index_track_vertex])-1e-9)

In [130]:
all_track_vertex_defaults.append(get_low_level_defaults_info(18))

track_0_QOVERPTHETA

[-5.67441e-09, 3.5582e-09, -5.67441e-09, -5.67441e-09, -5.67441e-09]


track_1_QOVERPTHETA

[-1000.0, 3.5582e-09, -1000.0, -5.43631e-09, -5.43631e-09]


track_2_QOVERPTHETA

[-1000.0, 3.5582e-09, -1000.0, -5.67441e-09, -5.67441e-09]


track_3_QOVERPTHETA

[-1000.0, 3.5582e-09, -1000.0, -4.35869e-09, -4.35869e-09]


track_4_QOVERPTHETA

[-1000.0, 3.5582e-09, -1000.0, -5.43631e-09, -5.43631e-09]


track_5_QOVERPTHETA

[-1000.0, 3.5582e-09, -1000.0, -4.35869e-09, -4.35869e-09]


track_6_QOVERPTHETA

[-1000.0, 3.5582e-09, -1000.0, -5.43631e-09, -5.43631e-09]


track_7_QOVERPTHETA

[-1000.0, 3.5582e-09, -1000.0, -4.35869e-09, -4.35869e-09]


track_8_QOVERPTHETA

[-1000.0, 3.5582e-09, -1000.0, -3.84411e-09, -3.84411e-09]


track_9_QOVERPTHETA

[-1000.0, 3.5582e-09, -1000.0, -4.35869e-09, -4.35869e-09]


track_10_QOVERPTHETA

[-1000.0, 3.5582e-09, -1000.0, -4.35869e-09, -4.35869e-09]


track_11_QOVERPTHETA

[-1000.0, 3.5582e-09, -1000.0, -4.35869e-09, -4.35869e-09]


trac

I would say keep, negative values means that 0 is not a viable option here (convariance can be negative for non-diagonal elements I guess).

In [174]:
feature_index_track_vertex = 18
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])
#np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [333]:
feature_index_track_vertex = 18
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.array(all_track_vertex_defaults[feature_index_track_vertex])-1e-11)

In [131]:
all_track_vertex_defaults.append(get_low_level_defaults_info(19))

track_0_QOVERPQOVERP

[2.28608e-08, 1.02578e-06, 2.28608e-08, 2.28608e-08, 2.28608e-08]


track_1_QOVERPQOVERP

[-1000.0, 1.02578e-06, -1000.0, 2.37754e-08, 2.37754e-08]


track_2_QOVERPQOVERP

[-1000.0, 1.02578e-06, -1000.0, 2.34429e-08, 2.34429e-08]


track_3_QOVERPQOVERP

[-1000.0, 1.02578e-06, -1000.0, 3.24699e-08, 3.24699e-08]


track_4_QOVERPQOVERP

[-1000.0, 1.02578e-06, -1000.0, 3.24699e-08, 3.24699e-08]


track_5_QOVERPQOVERP

[-1000.0, 1.02578e-06, -1000.0, 3.24699e-08, 3.24699e-08]


track_6_QOVERPQOVERP

[-1000.0, 1.02578e-06, -1000.0, 3.24276e-08, 3.24276e-08]


track_7_QOVERPQOVERP

[-1000.0, 1.02578e-06, -1000.0, 3.24699e-08, 3.24699e-08]


track_8_QOVERPQOVERP

[-1000.0, 1.02578e-06, -1000.0, 3.24699e-08, 3.24699e-08]


track_9_QOVERPQOVERP

[-1000.0, 1.02578e-06, -1000.0, 3.24699e-08, 3.24699e-08]


track_10_QOVERPQOVERP

[-1000.0, 1.02578e-06, -1000.0, 3.24699e-08, 3.24699e-08]


track_11_QOVERPQOVERP

[-1000.0, 1.02578e-06, -1000.0, 6.14995e-08, 6.14995e-08]


track_

Diagonal --> use 0.

In [175]:
feature_index_track_vertex = 19
#np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [297]:
feature_index_track_vertex = 19
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [132]:
all_track_vertex_defaults.append(get_low_level_defaults_info(20))

track_0_weight

[1.00008e-08, 0.989012, 1.00008e-08, 1.00008e-08, 1.00008e-08]


track_1_weight

[-1000.0, 0.989013, -1000.0, 1.00004e-08, 1.00004e-08]


track_2_weight

[-1000.0, 0.989013, -1000.0, 1.00022e-08, 1.00022e-08]


track_3_weight

[-1000.0, 0.989011, -1000.0, 1.00007e-08, 1.00007e-08]


track_4_weight

[-1000.0, 0.989011, -1000.0, 1.00028e-08, 1.00028e-08]


track_5_weight

[-1000.0, 0.98901, -1000.0, 1.00018e-08, 1.00018e-08]


track_6_weight

[-1000.0, 0.989006, -1000.0, 1.00071e-08, 1.00071e-08]


track_7_weight

[-1000.0, 0.988997, -1000.0, 1.00146e-08, 1.00146e-08]


track_8_weight

[-1000.0, 0.988983, -1000.0, 1.00431e-08, 1.00431e-08]


track_9_weight

[-1000.0, 0.988926, -1000.0, 1.00081e-08, 1.00081e-08]


track_10_weight

[-1000.0, 0.988873, -1000.0, 1.00999e-08, 1.00999e-08]


track_11_weight

[-1000.0, 0.988715, -1000.0, 1.02124e-08, 1.02124e-08]


track_12_weight

[-1000.0, 0.988199, -1000.0, 1.0028e-08, 1.0028e-08]


track_13_weight

[-1000.0, 0.988254, -1000.

Use 0 (most logical choice).

In [176]:
feature_index_track_vertex = 20
#np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [298]:
feature_index_track_vertex = 20
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [133]:
all_track_vertex_defaults.append(get_low_level_defaults_info(21))

track_0_vertex_mass

[0.13957, 34.9879, 0.13957, 0.13957, 0.13957]


track_1_vertex_mass

[-1000.0, 53.6423, -1000.0, 0.13957, 0.13957]


track_2_vertex_mass

[-1000.0, 53.6423, -1000.0, 0.13957, 0.13957]


track_3_vertex_mass

[-1000.0, 53.6423, -1000.0, 0.13957, 0.13957]


track_4_vertex_mass

[-1000.0, 53.6423, -1000.0, 0.13957, 0.13957]


track_5_vertex_mass

[-1000.0, 53.6423, -1000.0, 0.13957, 0.13957]


track_6_vertex_mass

[-1000.0, 53.6423, -1000.0, 0.13957, 0.13957]


track_7_vertex_mass

[-1000.0, 53.6423, -1000.0, 0.13957, 0.13957]


track_8_vertex_mass

[-1000.0, 53.6423, -1000.0, 0.13957, 0.13957]


track_9_vertex_mass

[-1000.0, 53.6423, -1000.0, 0.13957, 0.13957]


track_10_vertex_mass

[-1000.0, 53.6423, -1000.0, 0.13957, 0.13957]


track_11_vertex_mass

[-1000.0, 53.6423, -1000.0, 0.279191, 0.279191]


track_12_vertex_mass

[-1000.0, 53.6423, -1000.0, 0.13957, 0.13957]


track_13_vertex_mass

[-1000.0, 53.6423, -1000.0, 0.279646, 0.279646]


track_14_vertex_mass

[-10

Not sure: 0 would be a bit too far for some, but would make sense for most (mimic "no sv").  
Yes, using 0 sounds like the best option.

In [177]:
feature_index_track_vertex = 21
#np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [299]:
feature_index_track_vertex = 21
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [134]:
all_track_vertex_defaults.append(get_low_level_defaults_info(22))

track_0_vertex_displacement

[0.0, 20867.6, 0.0, 0.0, 0.0]


track_1_vertex_displacement

[-1000.0, 18267.1, -1000.0, 0.0, 0.0]


track_2_vertex_displacement

[-1000.0, 18920.2, -1000.0, 0.0, 0.0]


track_3_vertex_displacement

[-1000.0, 20867.6, -1000.0, 0.0, 0.0]


track_4_vertex_displacement

[-1000.0, 9759.45, -1000.0, 0.0, 0.0]


track_5_vertex_displacement

[-1000.0, 18267.1, -1000.0, 0.0, 0.0]


track_6_vertex_displacement

[-1000.0, 12596.3, -1000.0, 0.0, 0.0]


track_7_vertex_displacement

[-1000.0, 4051.43, -1000.0, 0.0, 0.0]


track_8_vertex_displacement

[-1000.0, 2638.61, -1000.0, 0.0, 0.0]


track_9_vertex_displacement

[-1000.0, 14377.6, -1000.0, 0.0, 0.0]


track_10_vertex_displacement

[-1000.0, 2782.33, -1000.0, 0.0, 0.0]


track_11_vertex_displacement

[-1000.0, 3404.91, -1000.0, 0.0, 0.0]


track_12_vertex_displacement

[-1000.0, 1538.25, -1000.0, 0.0, 0.0]


track_13_vertex_displacement

[-1000.0, 574.629, -1000.0, 0.0, 0.0]


track_14_vertex_displacement

[-1000.0

Use 0.

In [178]:
feature_index_track_vertex = 22
#np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

0 would interfer with true zeros

In [300]:
feature_index_track_vertex = 22
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33)-0.001)

In [135]:
all_track_vertex_defaults.append(get_low_level_defaults_info(23))

track_0_vertex_delta_eta_jet

[nan, nan, -999.0, -8.11161, -8.11161]


track_1_vertex_delta_eta_jet

[nan, nan, -1000.0, -8.11161, -8.11161]


track_2_vertex_delta_eta_jet

[nan, nan, -1000.0, -7.57314, -7.57314]


track_3_vertex_delta_eta_jet

[nan, nan, -1000.0, -8.10795, -8.10795]


track_4_vertex_delta_eta_jet

[nan, nan, -1000.0, -8.10795, -8.10795]


track_5_vertex_delta_eta_jet

[nan, nan, -1000.0, -8.11161, -8.11161]


track_6_vertex_delta_eta_jet

[nan, nan, -1000.0, -6.98585, -6.98585]


track_7_vertex_delta_eta_jet

[nan, nan, -1000.0, -7.08416, -7.08416]


track_8_vertex_delta_eta_jet

[nan, nan, -1000.0, -7.26977, -7.26977]


track_9_vertex_delta_eta_jet

[nan, nan, -1000.0, -7.24468, -7.24468]


track_10_vertex_delta_eta_jet

[nan, nan, -1000.0, -6.31929, -6.31929]


track_11_vertex_delta_eta_jet

[nan, nan, -1000.0, -5.91396, -5.91396]


track_12_vertex_delta_eta_jet

[nan, nan, -1000.0, -5.68993, -5.68993]


track_13_vertex_delta_eta_jet

[nan, nan, -1000.0, -4.96115, -

Keep as they are.

In [179]:
feature_index_track_vertex = 23
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])
#np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [301]:
feature_index_track_vertex = 23
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.array(all_track_vertex_defaults[feature_index_track_vertex])-0.001)

In [136]:
all_track_vertex_defaults.append(get_low_level_defaults_info(24))

track_0_vertex_delta_phi_jet

[nan, nan, -999.0, -3.14159, -3.14159]


track_1_vertex_delta_phi_jet

[nan, nan, -1000.0, -3.14159, -3.14159]


track_2_vertex_delta_phi_jet

[nan, nan, -1000.0, -3.14159, -3.14159]


track_3_vertex_delta_phi_jet

[nan, nan, -1000.0, -3.14159, -3.14159]


track_4_vertex_delta_phi_jet

[nan, nan, -1000.0, -3.14159, -3.14159]


track_5_vertex_delta_phi_jet

[nan, nan, -1000.0, -3.14158, -3.14158]


track_6_vertex_delta_phi_jet

[nan, nan, -1000.0, -3.14159, -3.14159]


track_7_vertex_delta_phi_jet

[nan, nan, -1000.0, -3.14157, -3.14157]


track_8_vertex_delta_phi_jet

[nan, nan, -1000.0, -3.14155, -3.14155]


track_9_vertex_delta_phi_jet

[nan, nan, -1000.0, -3.14158, -3.14158]


track_10_vertex_delta_phi_jet

[nan, nan, -1000.0, -3.14125, -3.14125]


track_11_vertex_delta_phi_jet

[nan, nan, -1000.0, -3.14153, -3.14153]


track_12_vertex_delta_phi_jet

[nan, nan, -1000.0, -3.14153, -3.14153]


track_13_vertex_delta_phi_jet

[nan, nan, -1000.0, -3.14125, -

Keep as they are.

In [180]:
feature_index_track_vertex = 24
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])
#np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [302]:
feature_index_track_vertex = 24
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.array(all_track_vertex_defaults[feature_index_track_vertex])-0.001)

In [137]:
all_track_vertex_defaults.append(get_low_level_defaults_info(25))

track_0_vertex_displacement_significance

[nan, nan, -999.0, 0.0, 0.0]


track_1_vertex_displacement_significance

[nan, nan, -1000.0, 0.0, 0.0]


track_2_vertex_displacement_significance

[nan, nan, -1000.0, 0.0, 0.0]


track_3_vertex_displacement_significance

[nan, nan, -1000.0, 0.0, 0.0]


track_4_vertex_displacement_significance

[nan, nan, -1000.0, 0.0, 0.0]


track_5_vertex_displacement_significance

[nan, nan, -1000.0, 0.0, 0.0]


track_6_vertex_displacement_significance

[nan, nan, -1000.0, 0.0, 0.0]


track_7_vertex_displacement_significance

[nan, nan, -1000.0, 0.0, 0.0]


track_8_vertex_displacement_significance

[-1000.0, 512.271, -1000.0, 0.0, 0.0]


track_9_vertex_displacement_significance

[-1000.0, 805.595, -1000.0, 0.0, 0.0]


track_10_vertex_displacement_significance

[-1000.0, 291.977, -1000.0, 0.0, 0.0]


track_11_vertex_displacement_significance

[-1000.0, 272.811, -1000.0, 0.0, 0.0]


track_12_vertex_displacement_significance

[-1000.0, 196.566, -1000.0, 0.0, 0.0

Use 0.

In [181]:
feature_index_track_vertex = 25
#np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

0 would interfer

In [303]:
feature_index_track_vertex = 25
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33)-0.001)

In [138]:
all_track_vertex_defaults.append(get_low_level_defaults_info(26))

track_0_vertex_n_tracks

[1.0, 27.0, 1.0, 1.0, 1.0]


track_1_vertex_n_tracks

[-1000.0, 29.0, -1000.0, 1.0, 1.0]


track_2_vertex_n_tracks

[-1000.0, 29.0, -1000.0, 1.0, 1.0]


track_3_vertex_n_tracks

[-1000.0, 29.0, -1000.0, 1.0, 1.0]


track_4_vertex_n_tracks

[-1000.0, 29.0, -1000.0, 1.0, 1.0]


track_5_vertex_n_tracks

[-1000.0, 29.0, -1000.0, 1.0, 1.0]


track_6_vertex_n_tracks

[-1000.0, 29.0, -1000.0, 1.0, 1.0]


track_7_vertex_n_tracks

[-1000.0, 29.0, -1000.0, 1.0, 1.0]


track_8_vertex_n_tracks

[-1000.0, 29.0, -1000.0, 1.0, 1.0]


track_9_vertex_n_tracks

[-1000.0, 29.0, -1000.0, 1.0, 1.0]


track_10_vertex_n_tracks

[-1000.0, 29.0, -1000.0, 1.0, 1.0]


track_11_vertex_n_tracks

[-1000.0, 28.0, -1000.0, 2.0, 2.0]


track_12_vertex_n_tracks

[-1000.0, 29.0, -1000.0, 1.0, 1.0]


track_13_vertex_n_tracks

[-1000.0, 29.0, -1000.0, 2.0, 2.0]


track_14_vertex_n_tracks

[-1000.0, 29.0, -1000.0, 2.0, 2.0]


track_15_vertex_n_tracks

[-1000.0, 29.0, -1000.0, 2.0, 2.0]


track_16_v

Should be 0, I guess.

In [182]:
feature_index_track_vertex = 26
#np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [304]:
feature_index_track_vertex = 26
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [139]:
all_track_vertex_defaults.append(get_low_level_defaults_info(27))

track_0_vertex_energy_fraction

[0.00432744, 1.01357, 0.00432744, 0.00432744, 0.00432744]


track_1_vertex_energy_fraction

[-1000.0, 1.01357, -1000.0, 0.0056943, 0.0056943]


track_2_vertex_energy_fraction

[-1000.0, 1.01016, -1000.0, 0.00491715, 0.00491715]


track_3_vertex_energy_fraction

[-1000.0, 1.00861, -1000.0, 0.00515934, 0.00515934]


track_4_vertex_energy_fraction

[-1000.0, 1.00599, -1000.0, 0.00666941, 0.00666941]


track_5_vertex_energy_fraction

[-1000.0, 1.00476, -1000.0, 0.00524205, 0.00524205]


track_6_vertex_energy_fraction

[-1000.0, 1.00345, -1000.0, 0.00970007, 0.00970007]


track_7_vertex_energy_fraction

[-1000.0, 1.0023, -1000.0, 0.00432744, 0.00432744]


track_8_vertex_energy_fraction

[-1000.0, 1.00207, -1000.0, 0.0109139, 0.0109139]


track_9_vertex_energy_fraction

[-1000.0, 1.00028, -1000.0, 0.00949435, 0.00949435]


track_10_vertex_energy_fraction

[-1000.0, 1.00097, -1000.0, 0.0153175, 0.0153175]


track_11_vertex_energy_fraction

[-1000.0, 1.00002, -1

Should be 0, I guess.

In [183]:
feature_index_track_vertex = 27
#np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), all_track_vertex_defaults[feature_index_track_vertex])
np.save(defaults_path+'custom_minima_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

In [305]:
feature_index_track_vertex = 27
np.save(defaults_path+'custom_defaults_track_vertex_var_{}.npy'.format(feature_index_track_vertex), np.zeros(33))

Defaults that are _just_ above 0 can be safely set to 0, all others I would probably keep as they are.

Build array with all defaults / minima (which can be indexed based on the full set of variables).

In [ ]:
import os
default_files = os.listdir(defaults_path)
default_files

In [310]:
custom_minima_files = []
for a in default_files:
    if 'custom_mi' in a:
        custom_minima_files.append(a)
custom_minima_files.sort()

In [311]:
custom_minima_files

['custom_minima_a4.npy',
 'custom_minima_a5.npy',
 'custom_minima_track_vertex_var_0.npy',
 'custom_minima_track_vertex_var_1.npy',
 'custom_minima_track_vertex_var_10.npy',
 'custom_minima_track_vertex_var_11.npy',
 'custom_minima_track_vertex_var_12.npy',
 'custom_minima_track_vertex_var_13.npy',
 'custom_minima_track_vertex_var_14.npy',
 'custom_minima_track_vertex_var_15.npy',
 'custom_minima_track_vertex_var_16.npy',
 'custom_minima_track_vertex_var_17.npy',
 'custom_minima_track_vertex_var_18.npy',
 'custom_minima_track_vertex_var_19.npy',
 'custom_minima_track_vertex_var_2.npy',
 'custom_minima_track_vertex_var_20.npy',
 'custom_minima_track_vertex_var_21.npy',
 'custom_minima_track_vertex_var_22.npy',
 'custom_minima_track_vertex_var_23.npy',
 'custom_minima_track_vertex_var_24.npy',
 'custom_minima_track_vertex_var_25.npy',
 'custom_minima_track_vertex_var_26.npy',
 'custom_minima_track_vertex_var_27.npy',
 'custom_minima_track_vertex_var_3.npy',
 'custom_minima_track_vertex_v

In [312]:
sorted_custom_minima_files = ['custom_minimum_a{}.npy'.format(j) for j in range(1,4)]
b = ['custom_minima_a{}.npy'.format(k) for k in range(4,6)]
c = ['custom_minima_track_vertex_var_{}.npy'.format(l) for l in range(0,28)]
sorted_custom_minima_files.extend(b)
sorted_custom_minima_files.extend(c)
sorted_custom_minima_files

['custom_minimum_a1.npy',
 'custom_minimum_a2.npy',
 'custom_minimum_a3.npy',
 'custom_minima_a4.npy',
 'custom_minima_a5.npy',
 'custom_minima_track_vertex_var_0.npy',
 'custom_minima_track_vertex_var_1.npy',
 'custom_minima_track_vertex_var_2.npy',
 'custom_minima_track_vertex_var_3.npy',
 'custom_minima_track_vertex_var_4.npy',
 'custom_minima_track_vertex_var_5.npy',
 'custom_minima_track_vertex_var_6.npy',
 'custom_minima_track_vertex_var_7.npy',
 'custom_minima_track_vertex_var_8.npy',
 'custom_minima_track_vertex_var_9.npy',
 'custom_minima_track_vertex_var_10.npy',
 'custom_minima_track_vertex_var_11.npy',
 'custom_minima_track_vertex_var_12.npy',
 'custom_minima_track_vertex_var_13.npy',
 'custom_minima_track_vertex_var_14.npy',
 'custom_minima_track_vertex_var_15.npy',
 'custom_minima_track_vertex_var_16.npy',
 'custom_minima_track_vertex_var_17.npy',
 'custom_minima_track_vertex_var_18.npy',
 'custom_minima_track_vertex_var_19.npy',
 'custom_minima_track_vertex_var_20.npy',


In [313]:
all_the_mins = np.load(defaults_path+sorted_custom_minima_files[0])
#a4_mins_ = np.load(defaults_path+sorted_custom_minima_files[3])
#all_the_mins = np.append(all_the_mins,a4_mins_)
#a5_mins_ = np.load(defaults_path+sorted_custom_minima_files[4])
#all_the_mins = np.append(all_the_mins,a5_mins_)
for k in range(1,len(sorted_custom_minima_files)):
    all_the_mins = np.append(all_the_mins, np.load(defaults_path+sorted_custom_minima_files[k]))

In [314]:
all_the_mins

array([ 2.29141e+01, -2.50000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,
        0.00000e+00, -7.92794e+00, -7.99466e+00, -7.99902e+00,
       -7.99968e+00, -7.99789e+00, -7.99928e+00, -7.99755e+00,
       -7.99724e+00, -7.99764e+00, -7.98958e+00, -7.99895e+00,
       -7.99545e+00, -7.96536e+00, -7.95577e+00, -7.82211e+00,
       -7.52736e+00, -6.92621e+00, -7.83785e+00, -7.48624e+00,
       -6.32885e+00, -7.59735e+00, -4.92800e+00, -7.38105e+00,
       -1.69054e+00, -1.94417e+00, -1.36310e+00, -5.92415e+00,
       -8.38711e-02, -7.44234e-02, -7.46253e-02, -2.07399e-01,
       -6.60398e-01, -3.44903e-01, -3.93904e+02, -1.85665e+02,
       -2.53083e+02, -2.37928e+02, -2.09215e+02, -2.72432e+02,
       -2.14421e+02, -2.82324e+02, -1.08551e+02, -1.84000e+02,
       -2.38852e+02, -1.19193e+02, -6.44819e+01, -4.371

In [315]:
len(all_the_mins)

941

Store all custom minima in the defaults directory, for easy look-up later.

In [316]:
np.save(defaults_path+'all_mins.npy', all_the_mins)

Make sure that the used defaults are not the minimum, if the minimum was already occupied by a decent fraction of meaningful samples...

In [318]:
sorted_custom_default_files = ['custom_default_a{}.npy'.format(j) for j in range(1,4)]
b = ['custom_defaults_a{}.npy'.format(k) for k in range(4,6)]
c = ['custom_defaults_track_vertex_var_{}.npy'.format(l) for l in range(0,28)]
sorted_custom_default_files.extend(b)
sorted_custom_default_files.extend(c)
sorted_custom_default_files

['custom_default_a1.npy',
 'custom_default_a2.npy',
 'custom_default_a3.npy',
 'custom_defaults_a4.npy',
 'custom_defaults_a5.npy',
 'custom_defaults_track_vertex_var_0.npy',
 'custom_defaults_track_vertex_var_1.npy',
 'custom_defaults_track_vertex_var_2.npy',
 'custom_defaults_track_vertex_var_3.npy',
 'custom_defaults_track_vertex_var_4.npy',
 'custom_defaults_track_vertex_var_5.npy',
 'custom_defaults_track_vertex_var_6.npy',
 'custom_defaults_track_vertex_var_7.npy',
 'custom_defaults_track_vertex_var_8.npy',
 'custom_defaults_track_vertex_var_9.npy',
 'custom_defaults_track_vertex_var_10.npy',
 'custom_defaults_track_vertex_var_11.npy',
 'custom_defaults_track_vertex_var_12.npy',
 'custom_defaults_track_vertex_var_13.npy',
 'custom_defaults_track_vertex_var_14.npy',
 'custom_defaults_track_vertex_var_15.npy',
 'custom_defaults_track_vertex_var_16.npy',
 'custom_defaults_track_vertex_var_17.npy',
 'custom_defaults_track_vertex_var_18.npy',
 'custom_defaults_track_vertex_var_19.npy'

In [319]:
all_the_defaults = np.load(defaults_path+sorted_custom_default_files[0])
for k in range(1,len(sorted_custom_default_files)):
    all_the_defaults = np.append(all_the_defaults, np.load(defaults_path+sorted_custom_default_files[k]))

In [320]:
all_the_defaults

array([ 0.00000000e+00, -3.00000000e+00, -1.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -1.00000000e-03,
       -1.00000000e-03,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -7.92894000e+00, -7.99566000e+00, -8.00002000e+00,
       -8.00068000e+00, -7.99889000e+00, -8.00028000e+00, -7.99855000e+00,
       -7.99824000e+00, -7.99864000e+00, -7.99058000e+00, -7.99995000e+00,
       -7.99645000e+00, -7.96636000e+00, -7.95677000e+00, -7.82311000e+00,
       -7.52836000e+00, -6.92721000e+00, -7.83885000e+00, -7.48724000e+00,
       -6.32985000e+00, -7.59835000e+00, -4.92900000e+00, -7.38205000e+00,
       -1.69154000e+00, -1.94517000e+00, -1.36410000e+00, -5.92515000e+00,
       -8.48711000e-02, -7.54234000e-02, -7.56253000e-02, -2.08399000e-01,
       -6.61398000e-01, -3.45903000e-01, -3.93905000e+02, -1.85666000e+02,
       -2.53084000e+02, -

In [321]:
len(all_the_defaults)

941

In [322]:
np.save(defaults_path+'all_defaults.npy', all_the_defaults)

In order to decide also at the level of scaled quantities whether some value is a default, it could help to scale the defaults themselves and store the resulting "scaled default" as well. The step is executed inside `clean_preprocess.ipynb`.  

The study showed that some defaults are too far off from the original distribution (non-diagonal elements of the covariance matrix basically), which means the scaled defaults are huge for variables with original values several orders of magnitude smaller than 1. Have to store the updated defaults! Keep the old ones for reference, though.

In [334]:
all_the_defaults_NEW = np.load(defaults_path+sorted_custom_default_files[0])
for k in range(1,len(sorted_custom_default_files)):
    all_the_defaults_NEW = np.append(all_the_defaults_NEW, np.load(defaults_path+sorted_custom_default_files[k]))

In [335]:
all_the_defaults_NEW

array([ 0.000000e+00, -3.000000e+00, -1.000000e+00,  0.000000e+00,
        0.000000e+00,  0.000000e+00,  0.000000e+00, -1.000000e-03,
       -1.000000e-03,  0.000000e+00,  0.000000e+00,  0.000000e+00,
        0.000000e+00,  0.000000e+00,  0.000000e+00,  0.000000e+00,
        0.000000e+00, -7.928940e+00, -7.995660e+00, -8.000020e+00,
       -8.000680e+00, -7.998890e+00, -8.000280e+00, -7.998550e+00,
       -7.998240e+00, -7.998640e+00, -7.990580e+00, -7.999950e+00,
       -7.996450e+00, -7.966360e+00, -7.956770e+00, -7.823110e+00,
       -7.528360e+00, -6.927210e+00, -7.838850e+00, -7.487240e+00,
       -6.329850e+00, -7.598350e+00, -4.929000e+00, -7.382050e+00,
       -1.691540e+00, -1.945170e+00, -1.364100e+00, -5.925150e+00,
       -8.487110e-02, -7.542340e-02, -7.562530e-02, -2.083990e-01,
       -6.613980e-01, -3.459030e-01, -3.939050e+02, -1.856660e+02,
       -2.530840e+02, -2.379290e+02, -2.092160e+02, -2.724330e+02,
       -2.144220e+02, -2.823250e+02, -1.085520e+02, -1.840010e

In [336]:
len(all_the_defaults_NEW)

941

In [338]:
np.save(defaults_path+'all_defaults_OLD.npy', all_the_defaults)
np.save(defaults_path+'all_defaults.npy', all_the_defaults_NEW)